In [ ]:
# 데이터 파일 준비
# 그냥 실행해서 사용 파일을 생성하도록 합니다.
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/train_04.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/test_04.csv')
train.head(2)
X_train = train.iloc[:, :-1]
Y_train = train[['ID', 'Segmentation']]
X_train.to_csv('X_train.csv', index=False)
Y_train.to_csv('Y_train.csv', index=False)
test.to_csv('X_test.csv', index=False)
print('파일을 준비했습니다!  X_train.csv, Y_train.csv, X_test.csv !')

파일을 준비했습니다!  X_train.csv, Y_train.csv, X_test.csv !


In [ ]:
# [0] 사용 라이브러리 import
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier  # 마지막주에 정리하고 사용하실 분만 하세요.
from lightgbm import LGBMClassifier  # 마지막주에 정리하고 사용하실 분만 하세요.
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)  # Accuracy 
    B = model.score(x_test, y_test)
    # A == B (적절), A >> B (과대적합), A, B값이 낮은 값 (과소적합)
    y_pred = model.predict(x_test)
    C = f1_score(y_test, y_pred, average='macro')  #  f1_score(정답, 예측)
    y_proba = model.predict_proba(x_test)
    D = roc_auc_score(y_test, y_proba, average='macro', multi_class='ovo')  # multi_class must be in ('ovo', 'ovr')
    # precision_score, recall_score # -> f1_score와 사용방법 동일
    return f'ACC : {A:.4f} {B:.4f} F1 : {C:.4f} AUC : {D:.4f}'

# [1] 파일 읽어오기
X = pd.read_csv('X_train.csv')
Y = pd.read_csv('Y_train.csv')
X_submission = pd.read_csv('X_test.csv')
#print(X.shape, Y.shape, X_submission.shape)  # (2154, 1)

# [3] X, X_submission을 연결
# - 행 삭제, 삽입, 이동이 있으면 안됨
# - 열(컬럼)에 대해 제약은 없음 (삭제, 삽입, 이동)
dfX = pd.concat([X, X_submission], axis=0)
#dfX.nunique()

# [3-1] Label Encoding
dfX['Gender'] = dfX['Gender'].astype('category').cat.codes
dfX['Ever_Married'] = dfX['Ever_Married'].astype('category').cat.codes
dfX['Graduated'] = dfX['Graduated'].astype('category').cat.codes
# [3-2] OneHot Encoding
dfX = pd.get_dummies(dfX)
#dfX.info()

# [4] 변수선택
dfX = dfX.drop(columns=['ID'])

# [5] 스케일링 (선택)
# 스케일링 결과는 ndarray 이기 때문에 사용시 ndarray의 특징, 사용법을 알고 있어야 함
# ndarray를 DataFrame으로 변경해서 사용하자!
dfX = pd.DataFrame(MinMaxScaler().fit_transform(dfX), columns=dfX.columns)
#print(type(dfX))  # <class 'numpy.ndarray'> // <class 'pandas.core.frame.DataFrame'>

# [6] dfX -> X, X_submission 분리
X_use = dfX.iloc[:6665, :]          # 모델링, 모델평가
X_submission = dfX.iloc[6665:, :]   # 제출용
Y_use = Y['Segmentation']
Y_use = Y_use - 1   # XGBClassifier
#print(Y_use.value_counts(normalize=True))

# [7] train, test 분리
temp = train_test_split(X_use, Y_use, test_size=0.2, random_state=4321, stratify=Y_use)
x_train, x_test, y_train, y_test = temp
#print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
#print(X_submission.shape)

# [8] 모델선택, 학습, 평가

#model1 = LogisticRegression(max_iter=1000)
#model1.fit(x_train, y_train)
#print(get_scores(model1, x_train, x_test, y_train, y_test))
# ACC : 0.5259 0.5244 F1 : 0.5096 AUC : 0.7653

#model2 = KNeighborsClassifier(12)
#model2.fit(x_train, y_train)
#print(get_scores(model2, x_train, x_test, y_train, y_test))

#model3 = DecisionTreeClassifier(max_depth=6, random_state=1)
#model3.fit(x_train, y_train)
#print(model3.get_depth())
#print(get_scores(model3, x_train, x_test, y_train, y_test))

#model4 = RandomForestClassifier(n_estimators=300, max_depth=6, random_state=1)
#model4.fit(x_train, y_train)
#print(get_scores(model4, x_train, x_test, y_train, y_test))

# XGBClassifier는  [1, 2, 3, 4], [-1, 1] 사용하지 못함
# [0, 1, 2, 3], [0, 1]와 같이 수정해 주어야 함
# 제출할 때는 [1, 2, 3, 4], [-1, 1] 로 복원해야 함
model5 = XGBClassifier(n_estimators=100, max_depth=3, random_state=123) 
model5.fit(x_train, y_train)
print(get_scores(model5, x_train, x_test, y_train, y_test))
#ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3], got [1 2 3 4]
#ACC : 0.6303 0.5439 F1 : 0.5331 AUC : 0.7855   n_estimators=50, max_depth=4

#model6 = LGBMClassifier(n_estimators=30, max_depth=3, random_state=123)   
#model6.fit(x_train, y_train)
#print(get_scores(model6, x_train, x_test, y_train, y_test))
# ACC : 0.5497 0.5439 F1 : 0.5330 AUC : 0.7916

# [9] 제출 파일 작성
submission = pd.DataFrame()
#submission['Segmentation'] = model6.predict(X_submission)
submission['Segmentation'] = model5.predict(X_submission) + 1  # Y_use = Y_use - 1 때문에
submission.to_csv('수험번호.csv', index=False)

# [10] 제출확인
#temp = pd.read_csv('수험번호.csv')
#print(temp.shape)
#print(temp['Segmentation'].unique())
#print(temp.head(5))

ACC : 0.6125 0.5394 F1 : 0.5281 AUC : 0.7919
